In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# import speicidfic models here
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb

import shap


In [16]:
target_column  = 'Mid_Price_Future'
window_size = '1min'

In [17]:
# load your dataset
def load_data():
    # df = pd.read_csv('data/lob/full_lob_head.csv', index_col=0)
    df = pd.read_parquet('data/output/full_normalised_lob_feature_set.parquet')
    print(df.info())
    return df

In [18]:
# preprocessing steps
def convert_timestamp(df):
    df['Timestamp'] = pd.to_numeric(df['Timestamp'])
    return df

def sort_values(df):
    df.sort_values(by=['Date','Timestamp'], inplace=True)
    return df

def pivot_table(df):
    df = df.pivot_table(index=['Timestamp', 'Exchange', 'Date'], 
                            columns='Order Type', 
                            values=['Price', 'Quantity'])
    df = df.reset_index()
    return df

def flatten_columns(df):
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    return df

def apply_window(df):
     # convert 'Timestamp' to a timedelta and add it to 'Date' to get a datetime
    df['Datetime'] = pd.to_datetime(df['Date']) + pd.to_timedelta(df['Timestamp'], unit='s')

    # set datetime as the index
    df.set_index('Datetime', inplace=True)

    # drop Exchange, Timestamp and Date columns
    df.drop(['Exchange', 'Timestamp', 'Date'], axis=1, inplace=True)

    df = df.resample(window_size).agg({
    'Price Ask': 'mean',
    'Price Bid': 'mean',
    'Quantity Ask': 'mean',
    'Quantity Bid': 'mean'
    })
    return df

def calculate_mid_price(df):
    df['Mid_Price'] = (df['Price Ask'] + df['Price Bid']) / 2
    return df

def market_price_spread(df):
    df['mean_market_spread'] = df['Price Ask'] - df['Price Bid']
    return df

def market_quantity_spread(df):
    df['mean_quantity_spread'] = df['Quantity Ask'] - df['Quantity Bid']
    return df

def create_lag_features(df):
    df['Mid_Price_Future'] = df['Mid_Price_EMA_Short_z-score_normalised'].shift(-20)
    # drop the last 20 rows
    df = df[:-20]
    df = df.dropna()
    df = df.drop(columns=['Timestamp', 'Exchange', 'Date', 'Bid', 'Ask'])
    return df

def split_data(df):
    X = df.drop(columns=[target_column])
    y = df[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

def preprocess_data(df):
    return (df.pipe(convert_timestamp)
              .pipe(sort_values)
            #   .pipe(pivot_table)
            #   .pipe(flatten_columns)
            #   .pipe(apply_window)
            #   .pipe(calculate_mid_price)
            #   .pipe(market_price_spread)
            #   .pipe(market_quantity_spread)
              .pipe(create_lag_features)
              .pipe(split_data))

In [19]:
# model pipeline
def test_model(model, X_train, X_test, y_train, y_test):
    # Define pipeline steps
    steps = [
        ('scaler', StandardScaler()),  # Feature scaling
        ('model', model)  # Classifier/regressor model
    ]
    
    # Create pipeline
    pipeline = Pipeline(steps)
    
    # Train the model
    pipeline.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = pipeline.predict(X_test)
    
    # Evaluate model performance
    # Calculate Mean Squared Error
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error: {mse}')
    # Calculate R^2 Score
    r2 = r2_score(y_test, y_pred)
    print(f'R^2 Score: {r2}')

    # # Create an explainer object
    # explainer = shap.Explainer(model)

    # # Calculate SHAP values
    # shap_values = explainer.shap_values(X_train)

    # # Plot the SHAP summary plot
    # shap.summary_plot(shap_values, X_train)
    
    # Additional evaluation metrics or visualizations can be added here
    
    # Return the trained pipeline
    return pipeline

In [20]:
# Main function to run the pipeline
def main():
    # load data
    data = load_data()
    # preprocess data
    X_train, X_test, y_train, y_test = preprocess_data(data)

    models = {
        # models go here
        # "Support Vector Machine": SVC(),
        # "Linear Regression": LinearRegression(),
        # "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
        "XGBoost": xgb.XGBRegressor()
    }
    
    # model testing
    for name, model in models.items():
        print("\nTesting", name)
        pipeline = test_model(model, X_train, X_test, y_train, y_test)
        
        # todo: Perform cross-validation
        # cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5)
        # print("Cross-validation scores:", cv_scores)
        
        # todo: Hyperparameter tuning using GridSearchCV
        # param_grid = {...}
        # grid_search = GridSearchCV(pipeline, param_grid, cv=5)
        # grid_search.fit(X_train, y_train)
        # best_params = grid_search.best_params_
        # print("Best hyperparameters:", best_params)
        
        # todo: Save the best model
        # best_model = grid_search.best_estimator_
        # Save the best_model for future use
        model.save_model('models/full-features.xgb')

In [21]:
main()

<class 'pandas.core.frame.DataFrame'>
Index: 40876791 entries, 3 to 41238692
Data columns (total 44 columns):
 #   Column                                         Dtype  
---  ------                                         -----  
 0   Timestamp                                      float64
 1   Exchange                                       object 
 2   Bid                                            object 
 3   Ask                                            object 
 4   Date                                           object 
 5   Mid_Price_Raw                                  float64
 6   Mid_Price_EMA_Raw                              float64
 7   Unnamed: 0_z-score_normalised                  float64
 8   Mid_Price_z-score_normalised                   float64
 9   Total_Order_Volume_z-score_normalised          float64
 10  OBV_z-score_normalised                         float64
 11  Total_Volume_Imbalance_z-score_normalised      float64
 12  Order_Imbalance_z-score_normalised           